# Intercepting Built-in Operation Attributes

`__getattr__` and `__getattribute__` for method-name attributes implicitly fetch by built-in operations, these methods may *not be run at all*. This means that operator overloading calls cannot be delegated to wrapped objects unless wrapper classes somehow redefine these methods themselves.

For example, attribute fetchs for the `__str__`, `__add__`, and `__getitem__` methods run implicitly by printing, + expressions, and indexing, respectively,are not routed to the generic attribute interception methods in 3.X. Specifically:

- In Python 3.X, *neither*, `__getattr__` nor `__getattribute__` is run for such attributes.
- In Python 2.X classic classes, `__getattr__` *is* run for such attributes if they are undefined in the class.
- In Python 2.X, `__getattribute__` is available for new-style classes only and works as it does in 3.X.

In all Python 3.X classes (and 2.X new-style classes), there is no direct way to generically intercept built-in operations like printing and addition.

Wrapper classes can work around this constraint by redefining all relevant operator overloading methods in the wrapper itself, in order to delegate calls. These extra methods can be added either manually, with tools, or by definition in and inheritance from common superclasses. This does, however, make object wrappers more work then they used to be when operator overloading methods are a part of a wrapped object's interface.

The following example, tests various attribute types and built-in operations on intances of classes containing `__getattr__` and `__getattribute__` methods:

In [3]:
class GetAttr:
    eggs = 88
    def __init__(self):             # eggs stored on class, spam on instance
        self.spam = 77
    def __len__(self):              # len here, else __getattr__ called with __len__
        print("__len__: 42")
    def __getattr__(self, attr):    # Provide __str__ if asked, else dummy func 
        print("Getattr: " + attr)
        if attr == "__str__":
            return lambda *args: "[Getattr str]"
        else:
            return lambda *args: None

In [1]:
class GetAttribute(object):         # object required in 2.X, implied in 3.X
    eggs = 88                       # In 2.X all are instances(object) auto
    def __init__(self):             # But must derive to get new-style tools
        self.spam = 77              # incl __getattribute__, some __X__ defaults
    def __len__(self):
        print("__len__: 42")
        return 42
    def __getattribute__(self, attr):
        if attr == "__str__":
            return lambda *args: '[Get attribute str]'
        else:
            return lambda *args: None

In [5]:
for Class in GetAttr, GetAttribute:
    print("\n" + Class.__name__.ljust(50, '='))
    
    X = Class()
    X.eggs              # Class attr
    X.spam              # Instance attr
    X.other             # Missing attr
    len(X)              # __len__ defined explicitly


GetAttr===========================================
Getattr: other
__len__: 42


TypeError: 'NoneType' object cannot be interpreted as an integer